<a href="https://colab.research.google.com/github/isdeniz/paper-24/blob/main/nlbse22_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
import re
import unicodedata as ud
from nltk.corpus import wordnet as wn
from nltk.corpus import words as w
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import os
pd.set_option('display.max_colwidth', None)

#Retrieving datasets

In [2]:
test = pd.read_csv('/content/drive/My Drive/nlbse22/github-labels-top3-803k-test.csv')
train = pd.read_csv('/content/drive/My Drive/nlbse22/github-labels-top3-803k-train.csv')
print('test:', test.shape, 'train:', train.shape)

test: (80518, 8) train: (722899, 8)


In [3]:
train.head()

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
0,0,https://api.github.com/repos/eamodio/vscode-gitlens/issues/1302,bug,2021-01-02T18:07:30Z,NONE,https://api.github.com/repos/eamodio/vscode-gitlens,Welcome screen on every editor window is very tedious,"I just discovered Gitlens and find the functionality useful, thank you to all who contribute.\r\n\r\nI have about a dozen editor windows open, and the install process added a Gitlens welcome tab to each and every one of them. Combined with the snowflake effect, all of the sudden VScode was consuming 300-400% cpu and my fan was raging, as soon as I hunted them all down everything was back to fine. The welcome note content is great (although putting it on _all_ the windows is a bit much, don't know how much control you have on that). But overall it was a bit of a sour first-use experience, just wanted to provide that feedback."
1,1,https://api.github.com/repos/binwiederhier/pcopy/issues/9,bug,2020-12-31T18:19:31Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"""pcopy invite"" and ""pcopy paste abc:"" does not check if clipboard exists",NaN
2,2,https://api.github.com/repos/binwiederhier/pcopy/issues/16,bug,2021-01-03T04:33:36Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"UI: Modal overlay is half transparent, shouldn't be",NaN
3,3,https://api.github.com/repos/Sothatsit/RoyalUrClient/issues/41,enhancement,2020-12-25T00:46:00Z,OWNER,https://api.github.com/repos/Sothatsit/RoyalUrClient,Make the loading screen scale with browser window size,"Currently the loading wheel is a fixed size in pixels, but it would be better to specify it in terms of percentage of the browser size."
4,4,https://api.github.com/repos/Malivil/TTT-Custom-Roles/issues/190,bug,2021-01-02T21:36:57Z,OWNER,https://api.github.com/repos/Malivil/TTT-Custom-Roles,Spectator - Investigate a way to strip weapons before they are spectating a player,To bring magneto stick floating


In [4]:
test.head()

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
0,6,https://api.github.com/repos/tlnagy/TIFF.jl/issues/7,enhancement,2020-04-07T09:08:50Z,NONE,https://api.github.com/repos/tlnagy/TIFF.jl,"ERROR: KeyError: key (TIFF.SAMPLEFORMAT_INT, 0x0008) not found","One more error might need to be caught.\r\n`4D-series.ome.tif` is sample file from [OME-TIFF website]( https://docs.openmicroscopy.org/ome-model/6.0.0/ome-tiff/data.html).\r\n\r\n```julia\r\njulia> TIFF.load(""4D-series.ome.tif"")\r\nERROR: KeyError: key (TIFF.SAMPLEFORMAT_INT, 0x0008) not found\r\nStacktrace:\r\n [1] getindex(::Dict{Tuple{TIFF.SampleFormats,Int64},DataType}, ::Tuple{TIFF.SampleFormats,UInt16}) at ./dict.jl:477\r\n [2] output(::TIFF.IFD{UInt32}) at /home/hf/.julia/dev/TIFF/src/ifds.jl:113\r\n [3] load(::String) at /home/hf/.julia/dev/TIFF/src/load.jl:14\r\n [4] top-level scope at REPL[2]:1\r\n```\r\n"
1,19,https://api.github.com/repos/tisboyo/Twitch_Bot/issues/57,enhancement,2020-11-27T07:17:21Z,OWNER,https://api.github.com/repos/tisboyo/Twitch_Bot,Add database backup to dropbox,NaN
2,25,https://api.github.com/repos/DrWhoCares/imgdanke/issues/70,enhancement,2021-01-02T19:35:34Z,OWNER,https://api.github.com/repos/DrWhoCares/imgdanke,Add a button/method to open the Source or Output folders,"Could also add a method to open up path to each file in the File list, by right-clicking.\r\nAlso an option to open up the path to the imgdanke.exe."
3,30,https://api.github.com/repos/DrWhoCares/imgdanke/issues/72,bug,2021-01-02T20:55:34Z,OWNER,https://api.github.com/repos/DrWhoCares/imgdanke,Processes are being started twice,At some point I refactored a few things and ended up leaving in an additional call to `process.Start()` that should've just been replaced by the `using Process process = Process.Start()` call.
4,54,https://api.github.com/repos/Bean-1/AOT/issues/3,bug,2020-12-29T15:34:35Z,OWNER,https://api.github.com/repos/Bean-1/AOT,Cannot add hp to wall,NaN


#Overview

**Distribution of labels**

Train

In [5]:
train['issue_label'].value_counts()

,count
issue_label,
bug,361103
enhancement,299374
question,62422


In [6]:
label_train = (train['issue_label'].value_counts(normalize=True) * 100).round(0)
label_train

,proportion
issue_label,
bug,50.0
enhancement,41.0
question,9.0


Test

In [7]:
test['issue_label'].value_counts()

,count
issue_label,
bug,40288
enhancement,33203
question,7027


In [8]:
label_test = (test['issue_label'].value_counts(normalize=True) * 100).round(0)
label_test

,proportion
issue_label,
bug,50.0
enhancement,41.0
question,9.0


Overall

In [9]:
combined = pd.concat([train, test], ignore_index = True)
combined.reset_index(drop = True, inplace = True)
print(combined.shape)

(803417, 8)


In [10]:
combined['issue_label'].value_counts()

,count
issue_label,
bug,401391
enhancement,332577
question,69449


In [11]:
label_combined = (combined['issue_label'].value_counts(normalize=True) * 100).round(0)
label_combined

,proportion
issue_label,
bug,50.0
enhancement,41.0
question,9.0


Number of repositories involved

In [12]:
combined.head(2)

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
0,0,https://api.github.com/repos/eamodio/vscode-gitlens/issues/1302,bug,2021-01-02T18:07:30Z,NONE,https://api.github.com/repos/eamodio/vscode-gitlens,Welcome screen on every editor window is very tedious,"I just discovered Gitlens and find the functionality useful, thank you to all who contribute.\r\n\r\nI have about a dozen editor windows open, and the install process added a Gitlens welcome tab to each and every one of them. Combined with the snowflake effect, all of the sudden VScode was consuming 300-400% cpu and my fan was raging, as soon as I hunted them all down everything was back to fine. The welcome note content is great (although putting it on _all_ the windows is a bit much, don't know how much control you have on that). But overall it was a bit of a sour first-use experience, just wanted to provide that feedback."
1,1,https://api.github.com/repos/binwiederhier/pcopy/issues/9,bug,2020-12-31T18:19:31Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"""pcopy invite"" and ""pcopy paste abc:"" does not check if clipboard exists",NaN


In [13]:
combined['repository_url'].nunique()

127595

In [14]:
combined['repository_name'] = combined['repository_url'].str.extract(r'https://api\.github\.com/repos/(.+)')

In [15]:
combined.head(2)

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body,repository_name
0,0,https://api.github.com/repos/eamodio/vscode-gitlens/issues/1302,bug,2021-01-02T18:07:30Z,NONE,https://api.github.com/repos/eamodio/vscode-gitlens,Welcome screen on every editor window is very tedious,"I just discovered Gitlens and find the functionality useful, thank you to all who contribute.\r\n\r\nI have about a dozen editor windows open, and the install process added a Gitlens welcome tab to each and every one of them. Combined with the snowflake effect, all of the sudden VScode was consuming 300-400% cpu and my fan was raging, as soon as I hunted them all down everything was back to fine. The welcome note content is great (although putting it on _all_ the windows is a bit much, don't know how much control you have on that). But overall it was a bit of a sour first-use experience, just wanted to provide that feedback.",eamodio/vscode-gitlens
1,1,https://api.github.com/repos/binwiederhier/pcopy/issues/9,bug,2020-12-31T18:19:31Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"""pcopy invite"" and ""pcopy paste abc:"" does not check if clipboard exists",NaN,binwiederhier/pcopy


In [16]:
combined['repository_name'].nunique()

127593

In [17]:
combined['repository_owner'] = combined['repository_name'].str.split('/', n = 1, expand = False).str[0]

In [18]:
combined.head(2)

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body,repository_name,repository_owner
0,0,https://api.github.com/repos/eamodio/vscode-gitlens/issues/1302,bug,2021-01-02T18:07:30Z,NONE,https://api.github.com/repos/eamodio/vscode-gitlens,Welcome screen on every editor window is very tedious,"I just discovered Gitlens and find the functionality useful, thank you to all who contribute.\r\n\r\nI have about a dozen editor windows open, and the install process added a Gitlens welcome tab to each and every one of them. Combined with the snowflake effect, all of the sudden VScode was consuming 300-400% cpu and my fan was raging, as soon as I hunted them all down everything was back to fine. The welcome note content is great (although putting it on _all_ the windows is a bit much, don't know how much control you have on that). But overall it was a bit of a sour first-use experience, just wanted to provide that feedback.",eamodio/vscode-gitlens,eamodio
1,1,https://api.github.com/repos/binwiederhier/pcopy/issues/9,bug,2020-12-31T18:19:31Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"""pcopy invite"" and ""pcopy paste abc:"" does not check if clipboard exists",NaN,binwiederhier/pcopy,binwiederhier


In [19]:
combined['repository_owner'].nunique()

85504

In [20]:
combined['repository_name'].isna().sum()

2

In [21]:
combined[combined['repository_name'].isna()]

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body,repository_name,repository_owner
237106,263728,https://arielvalentin.review-lab.github.com/api/v3/repos/lightstep/ls-trace-rb/issues/3,enhancement,2020-01-29T17:10:14Z,NONE,https://arielvalentin.review-lab.github.com/api/v3/repos/lightstep/ls-trace-rb,Add Backward Compatible Support with OpenTracing Request Propagation,Our existing services use LightStep for distributed tracing (http propagation with `ot-*` headers) and I would like to migrate to using `ls-trace` without having to upgrade all of our services to the latest version and also convert them to use `B3` propagation.\r\n\r\nIt would be helpful if the existing `HTTPPropagator` would also have support for LightStep's custom headers in addition to `B3` and `Datadog`:\r\n\r\nhttps://github.com/lightstep/ls-trace-rb/blob/86b1dd4896cf64d633957cfd87ad6bdf34fcbcb9/lib/ddtrace/propagation/http_propagator.rb#L14\r\n\r\nE.g. The `TEXT_MAP` propagator uses `DistributedHeaders` instead of the more restrictive `Headers` used by the `RACK_MAP` \r\nhttps://github.com/lightstep/ls-trace-rb/blob/86b1dd4896cf64d633957cfd87ad6bdf34fcbcb9/lib/ddtrace/opentracer/text_map_propagator.rb#L41\r\n\r\nhttps://github.com/lightstep/ls-trace-rb/blob/86b1dd4896cf64d633957cfd87ad6bdf34fcbcb9/lib/ddtrace/distributed_tracing/headers/datadog.rb#L22\r\n\r\nhttps://github.com/lightstep/ls-trace-rb/blob/86b1dd4896cf64d633957cfd87ad6bdf34fcbcb9/lib/ddtrace/opentracer/tracer.rb#L187\r\n\r\nThanks!,NaN,NaN
609423,677256,https://oneill38.review-lab.github.com/api/v3/repos/Oneill38/sit-test/issues/13,bug,2021-04-28T19:33:26Z,OWNER,https://oneill38.review-lab.github.com/api/v3/repos/Oneill38/sit-test,dagdasgasdg,### brew config\n\n![broken_top_color](https://user-images.githubusercontent.com/6576820/116462031-dc642280-a81d-11eb-98cb-42b05f9305c1.jpg)\r\n\r\n\n\n### brew doctor 3\n\nasdfad\n\n### brew doctor\n\n_No response_\n\n### What kinds of cats do you like?\n\n- [X] Orange cat (required. Everyone likes orange cats.)\n- [ ] Black cat,NaN,NaN


**Checking the nulls and types of information for each column**

issue_title

In [22]:
print('train:', train['issue_title'].isna().sum(), 'test:', test['issue_title'].isna().sum())

train: 0 test: 0


issue_body

In [23]:
print('train:', train['issue_body'].isna().sum(), 'test:', test['issue_body'].isna().sum())

train: 71872 test: 7916


issue_label

In [24]:
print('train:', train['issue_label'].isna().sum(), 'test:', test['issue_label'].isna().sum())

train: 0 test: 0


issue_created_at

In [25]:
print('train:', train['issue_created_at'].isna().sum(), 'test:', test['issue_created_at'].isna().sum())

train: 0 test: 0


issue_url

In [26]:
print('train:', train['issue_url'].isna().sum(), 'test:', test['issue_url'].isna().sum())

train: 0 test: 0


repository_url

In [27]:
print('train:', train['repository_url'].isna().sum(), 'test:', test['repository_url'].isna().sum())

train: 0 test: 0


issue_author_association

In [28]:
print('train:', train['issue_author_association'].isna().sum(), 'test:', test['issue_author_association'].isna().sum())

train: 0 test: 0


**Unique values under issue_author_association column**

In [29]:
combined['issue_author_association'].unique()

array(['NONE', 'OWNER', 'COLLABORATOR', 'CONTRIBUTOR', 'MEMBER',
       'MANNEQUIN'], dtype=object)

In [30]:
train['issue_author_association'].unique()

array(['NONE', 'OWNER', 'COLLABORATOR', 'CONTRIBUTOR', 'MEMBER',
       'MANNEQUIN'], dtype=object)

In [31]:
test['issue_author_association'].unique()

array(['NONE', 'OWNER', 'COLLABORATOR', 'MEMBER', 'CONTRIBUTOR'],
      dtype=object)

**Number of duplicate issues in the training and test sets**

In [32]:
train['issue_url'].duplicated().sum()

26220

In [33]:
test['issue_url'].duplicated().sum()

380

In [34]:
combined['issue_url'].duplicated().sum()

32026

In [35]:
duplicate_issue_urls = combined[combined['issue_url'].duplicated()]['issue_url']
train_duplicates = train[train['issue_url'].isin(duplicate_issue_urls)]
test_duplicates = test[test['issue_url'].isin(duplicate_issue_urls)]
cross_duplicates = pd.merge(train_duplicates, test_duplicates, on='issue_url', how='inner', suffixes=('_train', '_test'))

In [36]:
print(f'Number of duplicate issue URLs across both datasets: {len(cross_duplicates)}')
print('Duplicate issue URLs across both datasets:')
print(cross_duplicates['issue_url'].unique())

Number of duplicate issue URLs across both datasets: 10425
Duplicate issue URLs across both datasets:
['https://api.github.com/repos/matteobruni/tsparticles/issues/208'
 'https://api.github.com/repos/ankidroid/Anki-Android/issues/7526'
 'https://api.github.com/repos/anthonyamaro15/nv-credit-union/issues/28'
 ... 'https://api.github.com/repos/mpi2/PhenotypeData/issues/554'
 'https://api.github.com/repos/RENCI/ctmd/issues/421'
 'https://api.github.com/repos/ajency/hk_blog/issues/49']


In [37]:
train[train['issue_url'] == 'https://api.github.com/repos/matteobruni/tsparticles/issues/208']

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
146,155,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
58654,65217,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
59081,65700,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
89913,99942,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
91199,101356,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
135046,150120,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
357240,397136,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
579887,644390,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here
611443,679498,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here


In [38]:
test[test['issue_url'] == 'https://api.github.com/repos/matteobruni/tsparticles/issues/208']

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
25025,248227,https://api.github.com/repos/matteobruni/tsparticles/issues/208,enhancement,2020-04-03T15:29:16Z,OWNER,https://api.github.com/repos/matteobruni/tsparticles,Fluid output,I've seen this codepen https://codepen.io/matteobruni/pen/mdJYMQM and I think it's something portable here


In [39]:
cross_duplicates['issue_url'].nunique()

5426

**Issue Creation Time**

Train

In [40]:
train['datetime_column'] = pd.to_datetime(train['issue_created_at'])
max_value = train['datetime_column'].max()
min_value = train['datetime_column'].min()

print("Max datetime:", max_value)
print("Min datetime:", min_value)

Max datetime: 2021-05-31 23:35:26+00:00
Min datetime: 2005-11-03 14:27:00+00:00


Test

In [41]:
test['datetime_column'] = pd.to_datetime(test['issue_created_at'])
max_value = test['datetime_column'].max()
min_value = test['datetime_column'].min()

print("Max datetime:", max_value)
print("Min datetime:", min_value)

Max datetime: 2021-05-31 22:45:46+00:00
Min datetime: 2007-11-14 14:22:37+00:00


Overall

In [42]:
combined['datetime_column'] = pd.to_datetime(combined['issue_created_at'])
max_value = combined['datetime_column'].max()
min_value = combined['datetime_column'].min()

print("Max datetime:", max_value)
print("Min datetime:", min_value)

Max datetime: 2021-05-31 23:35:26+00:00
Min datetime: 2005-11-03 14:27:00+00:00
